In [5]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    print(path)
    data = load_files(path)
    print(data['target'])
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    print(dog_targets)
    return dog_files, dog_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('Udacity_Projects/Humpback_Whale_Identification/train1')
#valid_files, valid_targets = load_dataset('Udacity_Projects/Humpback_Whale_Identification/dogImages/valid')
test_files, test_targets = load_dataset('Udacity_Projects/Humpback_Whale_Identification/test')

# load list of dog names
whale_names = [item[20:-1] for item in sorted(glob("Udacity_Projects/Humpback_Whale_Identification/train/*/"))]

# print statistics about the dataset
print('There are %d total whale categories.' % len(whale_names))
#print('There are %s total whale images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training whale images.' % len(train_files))
#print('There are %d validation whale images.' % len(valid_files))
print('There are %d test whale images.'% len(test_files))

Udacity_Projects/Humpback_Whale_Identification/train1
[0 0 0 1 1 0 0 1 1 1 0 1 1 0 0 0 1 1]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
Udacity_Projects/Humpback_Whale_Identification/test
[]
[]
There are 0 total whale categories.
There are 18 training whale images.
There are 0 test whale images.


In [2]:
from collections import defaultdict
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os
import glob
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.models import Model
from keras.layers import Embedding, Flatten, Input, merge
from keras.optimizers import Adam
from keras.layers import Conv2D, MaxPooling2D, Input, Dense, Flatten, GlobalMaxPooling2D
from keras.models import Model
import glob
import os
from PIL import Image
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, \
    GlobalMaxPool2D, Concatenate, GlobalMaxPooling2D, GlobalAveragePooling2D, Lambda
from keras.applications.resnet50 import ResNet50
import pandas as pd
import numpy as np
import os
import glob
from sklearn.neighbors import NearestNeighbors

C:\Users\Ravi Mittal\Anaconda2\envs\Python35_Environment\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
class sample_gen(object):
    def __init__(self, file_class_mapping, other_class = "new_whale"):
        self.file_class_mapping= file_class_mapping
        self.class_to_list_files = defaultdict(list)
        self.list_other_class = []
        self.list_all_files = list(file_class_mapping.keys())
        self.range_all_files = list(range(len(self.list_all_files)))

        for file, class_ in file_class_mapping.items():
            if class_ == other_class:
                self.list_other_class.append(file)
            else:
                self.class_to_list_files[class_].append(file)

        self.list_classes = list(set(self.file_class_mapping.values()))
        self.range_list_classes= range(len(self.list_classes))
        self.class_weight = np.array([len(self.class_to_list_files[class_]) for class_ in self.list_classes])
        self.class_weight = self.class_weight/np.sum(self.class_weight)

    def get_sample(self):
        class_idx = np.random.choice(self.range_list_classes, 1, p=self.class_weight)[0]
        examples_class_idx = np.random.choice(range(len(self.class_to_list_files[self.list_classes[class_idx]])), 2)
        positive_example_1, positive_example_2 = \
            self.class_to_list_files[self.list_classes[class_idx]][examples_class_idx[0]],\
            self.class_to_list_files[self.list_classes[class_idx]][examples_class_idx[1]]


        negative_example = None
        while negative_example is None or self.file_class_mapping[negative_example] == \
                self.file_class_mapping[positive_example_1]:
            negative_example_idx = np.random.choice(self.range_all_files, 1)[0]
            negative_example = self.list_all_files[negative_example_idx]
        return positive_example_1, negative_example, positive_example_2




batch_size = 8
input_shape = (256, 256)
base_path = "Udacity_Projects/Humpback_Whale_Identification/train/"
def identity_loss(y_true, y_pred):

    return K.mean(y_pred - 0 * y_true)


def bpr_triplet_loss(X):

    positive_item_latent, negative_item_latent, user_latent = X

    # BPR loss
    loss = 1.0 - K.sigmoid(
        K.sum(user_latent * positive_item_latent, axis=-1, keepdims=True) -
        K.sum(user_latent * negative_item_latent, axis=-1, keepdims=True))

    return loss

def get_base_model():
    latent_dim = 50
    base_model = ResNet50(include_top=False) # use weights='imagenet' locally

    # for layer in base_model.layers:
    #     layer.trainable = False

    x = base_model.output
    x = GlobalMaxPooling2D()(x)
    x = Dropout(0.5)(x)
    dense_1 = Dense(latent_dim)(x)
    normalized = Lambda(lambda  x: K.l2_normalize(x,axis=1))(dense_1)
    base_model = Model(base_model.input, normalized, name="base_model")
    return base_model

def build_model():
    base_model = get_base_model()

    positive_example_1 = Input(input_shape+(3,) , name='positive_example_1')
    negative_example = Input(input_shape+(3,), name='negative_example')
    positive_example_2 = Input(input_shape+(3,), name='positive_example_2')

    positive_example_1_out = base_model(positive_example_1)
    negative_example_out = base_model(negative_example)
    positive_example_2_out = base_model(positive_example_2)

    loss = merge(
        [positive_example_1_out, negative_example_out, positive_example_2_out],
        mode=bpr_triplet_loss,
        name='loss',
        output_shape=(1, ))

    model = Model(
        input=[positive_example_1, negative_example, positive_example_2],
        output=loss)
    model.compile(loss=identity_loss, optimizer=Adam(0.000001))

    print(model.summary())

    return model


model_name = "triplet_model"

file_path = model_name + "weights.best.hdf5"



def build_inference_model(weight_path=file_path):
    base_model = get_base_model()

    positive_example_1 = Input(input_shape+(3,) , name='positive_example_1')
    negative_example = Input(input_shape+(3,), name='negative_example')
    positive_example_2 = Input(input_shape+(3,), name='positive_example_2')

    positive_example_1_out = base_model(positive_example_1)
    negative_example_out = base_model(negative_example)
    positive_example_2_out = base_model(positive_example_2)

    loss = merge(
        [positive_example_1_out, negative_example_out, positive_example_2_out],
        mode=bpr_triplet_loss,
        name='loss',
        output_shape=(1, ))

    model = Model(
        input=[positive_example_1, negative_example, positive_example_2],
        output=loss)
    model.compile(loss=identity_loss, optimizer=Adam(0.000001))

    model.load_weights(weight_path)

    inference_model = Model(base_model.get_input_at(0), output=base_model.get_output_at(0))
    inference_model.compile(loss="mse", optimizer=Adam(0.000001))
    print(inference_model.summary())

    return inference_model

def read_and_resize(filepath):
    im = Image.open((filepath)).convert('RGB')
    im = im.resize(input_shape)
    im_array = np.array(im, dtype="uint8")[..., ::-1]
    return np.array(im_array / (np.max(im_array)+ 0.001), dtype="float32")


def augment(im_array):
    if np.random.uniform(0, 1) > 0.9:
        im_array = np.fliplr(im_array)
    return im_array

def gen(triplet_gen):
    while True:
        list_positive_examples_1 = []
        list_negative_examples = []
        list_positive_examples_2 = []

        for i in range(batch_size):
            positive_example_1, negative_example, positive_example_2 = triplet_gen.get_sample()
            positive_example_1_img, negative_example_img, positive_example_2_img = read_and_resize(base_path+positive_example_1), \
                                                                       read_and_resize(base_path+negative_example), \
                                                                       read_and_resize(base_path+positive_example_2)

            positive_example_1_img, negative_example_img, positive_example_2_img = augment(positive_example_1_img), \
                                                                                   augment(negative_example_img), \
                                                                                   augment(positive_example_2_img)

            list_positive_examples_1.append(positive_example_1_img)
            list_negative_examples.append(negative_example_img)
            list_positive_examples_2.append(positive_example_2_img)

        list_positive_examples_1 = np.array(list_positive_examples_1)
        list_negative_examples = np.array(list_negative_examples)
        list_positive_examples_2 = np.array(list_positive_examples_2)
        yield [list_positive_examples_1, list_negative_examples, list_positive_examples_2], np.ones(batch_size)

In [5]:
num_epochs = 10

# Read data
data = pd.read_csv('Udacity_Projects/Humpback_Whale_Identification/train.csv')
train, test = train_test_split(data, test_size=0.3, shuffle=True, random_state=1337)
file_id_mapping_train = {k: v for k, v in zip(train.Image.values, train.Id.values)}
file_id_mapping_test = {k: v for k, v in zip(test.Image.values, test.Id.values)}
train_gen = sample_gen(file_id_mapping_train)
test_gen = sample_gen(file_id_mapping_test)



# Prepare the test triplets

#model = build_model()



#model.load_weights(file_path)

#checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

#early = EarlyStopping(monitor="val_loss", mode="min", patience=2)

#callbacks_list = [checkpoint, early]  # early

#history = model.fit_generator(gen(train_gen), validation_data=gen(test_gen), epochs=3, verbose=2, workers=4, 
#                              callbacks=callbacks_list, steps_per_epoch=300, validation_steps=30)
                              
                              
model_name = "triplet_loss"
def data_generator(fpaths, batch=16):
    i = 0
    for path in fpaths:
        if i == 0:
            imgs = []
            fnames = []
        i += 1
        img = read_and_resize(path)
        imgs.append(img)
        fnames.append(os.path.basename(path))
        if i == batch:
            i = 0
            imgs = np.array(imgs)
            yield fnames, imgs
    if i < batch:
        imgs = np.array(imgs)
        yield fnames, imgs
    raise StopIteration()

data = pd.read_csv('Udacity_Projects/Humpback_Whale_Identification/train.csv')

file_id_mapping = {k: v for k, v in zip(data.Image.values, data.Id.values)}

inference_model = build_inference_model()

train_files = glob.glob("Udacity_Projects/Humpback_Whale_Identification/train/*.jpg")
test_files = glob.glob("Udacity_Projects/Humpback_Whale_Identification/test/*.jpg")

train_preds = []
train_file_names = []
i = 1
for fnames, imgs in data_generator(train_files, batch=32):
    print(i*32/len(train_files)*100)
    i += 1
    predicts = inference_model.predict(imgs)
    predicts = predicts.tolist()
    train_preds += predicts
    train_file_names += fnames

train_preds = np.array(train_preds)

test_preds = []
test_file_names = []
i = 1
for fnames, imgs in data_generator(test_files, batch=32):
    print(i * 32 / len(test_files) * 100)
    i += 1
    predicts = inference_model.predict(imgs)
    predicts = predicts.tolist()
    test_preds += predicts
    test_file_names += fnames

test_preds = np.array(test_preds)

neigh = NearestNeighbors(n_neighbors=6)
neigh.fit(train_preds)
#distances, neighbors = neigh.kneighbors(train_preds)

#print(distances, neighbors)

distances_test, neighbors_test = neigh.kneighbors(test_preds)

distances_test, neighbors_test = distances_test.tolist(), neighbors_test.tolist()

preds_str = []

for filepath, distance, neighbour_ in zip(test_file_names, distances_test, neighbors_test):
    sample_result = []
    sample_classes = []
    for d, n in zip(distance, neighbour_):
        train_file = train_files[n].split(os.sep)[-1]
        class_train = file_id_mapping[train_file]
        sample_classes.append(class_train)
        sample_result.append((class_train, d))

    if "new_whale" not in sample_classes:
        sample_result.append(("new_whale", 0.1))
    sample_result.sort(key=lambda x: x[1])
    sample_result = sample_result[:5]
    preds_str.append(" ".join([x[0] for x in sample_result]))

df = pd.DataFrame(preds_str, columns=["Id"])
df['Image'] = [x.split(os.sep)[-1] for x in test_file_names]
df.to_csv("sub_%s.csv"%model_name, index=False)

C:\Users\Ravi Mittal\Anaconda2\envs\Python35_Environment\lib\site-packages\ipykernel_launcher.py:117: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
C:\Users\Ravi Mittal\Anaconda2\envs\Python35_Environment\lib\site-packages\keras\legacy\layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
C:\Users\Ravi Mittal\Anaconda2\envs\Python35_Environment\lib\site-packages\ipykernel_launcher.py:121: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("lo...)`
C:\Users\Ravi Mittal\Anaconda2\envs\Python35_Environment\lib\site-packages\ipykernel_launcher.py:126: UserWarning: Update your `Model` call to the Keras 2 API: `Model(Tensor("in..., outputs=Tensor("la...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

0.3248730964467005
0.649746192893401
0.9746192893401016
1.299492385786802
1.6243654822335025
1.9492385786802031
2.2741116751269037
2.598984771573604
2.9238578680203045
3.248730964467005
3.5736040609137056
3.8984771573604062
4.223350253807107
4.548223350253807
4.873096446700507
5.197969543147208
5.522842639593908
5.847715736040609
6.1725888324873095
6.49746192893401
6.82233502538071
7.147208121827411
7.472081218274111
7.7969543147208125
8.121827411167512
8.446700507614214
8.771573604060915
9.096446700507615
9.421319796954315
9.746192893401014
10.071065989847716
10.395939086294415
10.720812182741117
11.045685279187817
11.370558375634518
11.695431472081218
12.02030456852792
12.345177664974619
12.67005076142132
12.99492385786802
13.319796954314722
13.64467005076142
13.969543147208121
14.294416243654823
14.619289340101524
14.944162436548222
15.269035532994923
15.593908629441625
15.918781725888326
16.243654822335024
16.568527918781726
16.893401015228427
17.21827411167513
17.54314720812183
17

28.904548366431776
29.10954516335682
29.314541960281872
29.51953875720692
29.724535554131965
29.929532351057013
30.134529147982065
30.33952594490711
30.544522741832157
30.74951953875721
30.954516335682253
31.1595131326073
31.364509929532353
31.569506726457398
31.774503523382446
31.979500320307498
32.18449711723254
32.38949391415759
32.59449071108264
32.79948750800769
33.00448430493274
33.20948110185778
33.41447789878283
33.61947469570788
33.82447149263293
34.029468289557975
34.23446508648303
34.43946188340807
34.644458680333116
34.84945547725817
35.05445227418322
35.259449071108264
35.46444586803331
35.66944266495836
35.874439461883405
36.07943625880846
36.28443305573351
36.48942985265855
36.6944266495836
36.89942344650865
37.104420243433694
37.309417040358746
37.5144138372838
37.71941063420884
37.924407431133886
38.12940422805894
38.33440102498398
38.539397821909034
38.744394618834086
38.94939141575912
39.154388212684175
39.35938500960923
39.56438180653427
39.76937860345932
39.9743754

In [34]:
for i in range(1,10):
    print(i * 32 / len(test_files) * 100)

train_files
    

0.20499679692504805
0.4099935938500961
0.6149903907751442
0.8199871877001922
1.0249839846252402
1.2299807815502883
1.4349775784753362
1.6399743754003844
1.8449711723254325


['Udacity_Projects/Humpback_Whale_Identification/train\\00022e1a.jpg',
 'Udacity_Projects/Humpback_Whale_Identification/train\\000466c4.jpg',
 'Udacity_Projects/Humpback_Whale_Identification/train\\00087b01.jpg',
 'Udacity_Projects/Humpback_Whale_Identification/train\\001296d5.jpg',
 'Udacity_Projects/Humpback_Whale_Identification/train\\0014cfdf.jpg',
 'Udacity_Projects/Humpback_Whale_Identification/train\\0025e8c2.jpg',
 'Udacity_Projects/Humpback_Whale_Identification/train\\0026a8ab.jpg',
 'Udacity_Projects/Humpback_Whale_Identification/train\\0031c258.jpg',
 'Udacity_Projects/Humpback_Whale_Identification/train\\0035632e.jpg',
 'Udacity_Projects/Humpback_Whale_Identification/train\\0037e7d3.jpg',
 'Udacity_Projects/Humpback_Whale_Identification/train\\00389cd7.jpg',
 'Udacity_Projects/Humpback_Whale_Identification/train\\0042dcc4.jpg',
 'Udacity_Projects/Humpback_Whale_Identification/train\\0042ea34.jpg',
 'Udacity_Projects/Humpback_Whale_Identification/train\\00467ae9.jpg',
 'Udac

In [6]:
train_files12 = glob.glob("Udacity_Projects/Humpback_Whale_Identification/train/*.jpg")
train_files12

['Udacity_Projects/Humpback_Whale_Identification/train\\00022e1a.jpg',
 'Udacity_Projects/Humpback_Whale_Identification/train\\000466c4.jpg',
 'Udacity_Projects/Humpback_Whale_Identification/train\\00087b01.jpg',
 'Udacity_Projects/Humpback_Whale_Identification/train\\001296d5.jpg',
 'Udacity_Projects/Humpback_Whale_Identification/train\\0014cfdf.jpg',
 'Udacity_Projects/Humpback_Whale_Identification/train\\0025e8c2.jpg',
 'Udacity_Projects/Humpback_Whale_Identification/train\\0026a8ab.jpg',
 'Udacity_Projects/Humpback_Whale_Identification/train\\0031c258.jpg',
 'Udacity_Projects/Humpback_Whale_Identification/train\\0035632e.jpg',
 'Udacity_Projects/Humpback_Whale_Identification/train\\0037e7d3.jpg',
 'Udacity_Projects/Humpback_Whale_Identification/train\\00389cd7.jpg',
 'Udacity_Projects/Humpback_Whale_Identification/train\\0042dcc4.jpg',
 'Udacity_Projects/Humpback_Whale_Identification/train\\0042ea34.jpg',
 'Udacity_Projects/Humpback_Whale_Identification/train\\00467ae9.jpg',
 'Udac

In [39]:
distances_test

[[0.6253695607412527,
  0.6372356994000763,
  0.70264105714797,
  0.7061815057103469,
  0.7113486426185298,
  0.7188966539025937],
 [0.6059088149872566,
  0.6101320921565783,
  0.6249766883207775,
  0.6300097521169286,
  0.6365296517005681,
  0.6387397203222297],
 [0.6616080729444426,
  0.6845273452364837,
  0.7117892099273213,
  0.7117892099273213,
  0.7375481767503829,
  0.7388198326991688],
 [0.6096564552024257,
  0.6122194435894893,
  0.624664776412409,
  0.6327882772330948,
  0.6453383065583952,
  0.6488837081741082],
 [0.6414514607360157,
  0.6968587806604603,
  0.7023602038038999,
  0.7226405321378444,
  0.7326361788798482,
  0.7366223004152744],
 [0.6496156477483029,
  0.6546523239727368,
  0.6698418202054509,
  0.6715998676738574,
  0.6729274819877724,
  0.704533497529016],
 [0.0,
  0.5166397253791313,
  0.6419167855104224,
  0.6550694833452018,
  0.6622759334484447,
  0.6634316883601076],
 [0.6720695776995063,
  0.6768464244384265,
  0.6917664326208716,
  0.6943299110232446,


In [42]:
neighbors_test

[[8505, 7312, 873, 6023, 3266, 8385],
 [2860, 9755, 4550, 4521, 72, 994],
 [961, 6375, 5447, 6119, 3836, 1138],
 [8371, 6382, 3166, 3259, 6666, 5917],
 [9241, 9360, 6761, 1860, 5422, 3557],
 [92, 1694, 2939, 9578, 1864, 530],
 [3756, 1303, 293, 3092, 9191, 5284],
 [3008, 6987, 7176, 9196, 711, 277],
 [5806, 7947, 2831, 7541, 5645, 4794],
 [1654, 3170, 4393, 6266, 2462, 2534],
 [8434, 594, 6862, 4508, 5274, 6529],
 [5635, 5242, 8363, 1195, 2441, 3632],
 [9358, 616, 3425, 6262, 7518, 6040],
 [5032, 7967, 5058, 7035, 1414, 4598],
 [9420, 326, 2893, 5051, 8363, 9088],
 [3530, 7363, 9444, 4309, 5592, 9628],
 [3808, 507, 9794, 904, 6788, 5574],
 [7314, 9789, 2174, 7555, 139, 4242],
 [5952, 8517, 9695, 900, 5691, 4505],
 [9576, 6635, 1828, 6797, 8283, 6083],
 [8328, 3165, 7555, 2457, 2487, 7736],
 [8155, 4936, 7222, 6467, 8693, 8383],
 [2134, 4439, 9029, 1349, 4167, 6128],
 [1452, 1903, 5993, 1359, 91, 72],
 [6823, 2387, 6484, 4452, 2733, 8231],
 [6182, 9608, 8769, 7692, 2004, 5607],
 [2811, 

In [43]:
test_file_names

['00029b3a.jpg',
 '0003c693.jpg',
 '000bc353.jpg',
 '0010a672.jpg',
 '00119c3f.jpg',
 '001259cc.jpg',
 '0015f9b4.jpg',
 '0018c4ba.jpg',
 '001bf484.jpg',
 '002d8d81.jpg',
 '00342aa2.jpg',
 '00367faf.jpg',
 '003abee6.jpg',
 '003ac3da.jpg',
 '003be4bf.jpg',
 '003e42df.jpg',
 '004cd940.jpg',
 '0051ef38.jpg',
 '0056000b.jpg',
 '005d5f4f.jpg',
 '0067109a.jpg',
 '006affcc.jpg',
 '00701cff.jpg',
 '007b353c.jpg',
 '0087b1bf.jpg',
 '008a0615.jpg',
 '008be392.jpg',
 '00944069.jpg',
 '009a4db9.jpg',
 '009c3b60.jpg',
 '009fe2db.jpg',
 '00a5f43e.jpg',
 '00a716d3.jpg',
 '00ac0428.jpg',
 '00ae107b.jpg',
 '00aec561.jpg',
 '00b0d279.jpg',
 '00b436ee.jpg',
 '00b55e23.jpg',
 '00b75461.jpg',
 '00ba8a1f.jpg',
 '00c6a099.jpg',
 '00ce5ca3.jpg',
 '00dfd1df.jpg',
 '00e24ca4.jpg',
 '00e957d5.jpg',
 '00ec6537.jpg',
 '00f6144a.jpg',
 '00f8a2e9.jpg',
 '00fb828f.jpg',
 '00fbb200.jpg',
 '00ff0cb6.jpg',
 '01039b00.jpg',
 '0112cd97.jpg',
 '011ecf53.jpg',
 '011fc871.jpg',
 '0126c54c.jpg',
 '0128d810.jpg',
 '012acadb.jpg

In [9]:
train_files[2154].split(os.sep)[-1]

'37c40732.jpg'

In [50]:
test_preds

array([[-0.1403694 , -0.1350673 , -0.1142645 , ...,  0.14570779,
        -0.21124299,  0.19486624],
       [ 0.03344754, -0.01330293, -0.04765908, ..., -0.00492771,
        -0.20419708,  0.11683816],
       [-0.30195269, -0.06984662,  0.09374071, ..., -0.08801752,
        -0.00187489,  0.13779409],
       ...,
       [-0.01392007, -0.04933428,  0.11561102, ...,  0.03559744,
        -0.0639216 , -0.02377326],
       [-0.12385946, -0.00980061, -0.03373785, ...,  0.03561875,
        -0.00308782, -0.00534032],
       [ 0.05094155,  0.25367185,  0.1514273 , ..., -0.18842568,
        -0.0983234 , -0.01425343]])

In [27]:
j=1

for fnames, imgs in data_generator(test_files, batch=3):
    if j==1:
        #print(j * 32 / len(test_files) * 100)
        print(fnames)
        #print(imgs)
        j += 1    
        predicts = inference_model.predict(imgs)
        print(predicts)
        predicts = predicts.tolist()
        #print(predicts)
        #test_preds += predicts
        #test_file_names += fnames
        break
   

['00029b3a.jpg', '0003c693.jpg', '000bc353.jpg']
[[-0.1403694  -0.1350673  -0.1142645   0.06383383  0.1461542  -0.04797067
   0.29530352  0.16135003  0.04619833  0.13871168 -0.07192446  0.00223471
   0.02535738  0.16348086  0.13893452  0.02116495 -0.19789134  0.13141024
  -0.17551629 -0.03024921  0.03295987 -0.23345844 -0.18757668  0.3011653
   0.07286063  0.03850053  0.06306826 -0.00274378 -0.10328434  0.08378454
   0.11181185 -0.2243647   0.12092613  0.11403517  0.06911522 -0.11971541
  -0.00821793  0.20553973  0.10248771  0.02670185  0.12750742  0.29207528
  -0.13635412  0.12628064  0.04415873 -0.02834804 -0.17454366  0.14570779
  -0.21124296  0.19486625]
 [ 0.03344754 -0.01330293 -0.04765908 -0.18017027  0.00685519  0.05917708
   0.41384113 -0.05632648  0.13462691 -0.09483448  0.01243537  0.02186797
  -0.14690821  0.03311787  0.22834213 -0.04393258 -0.07579932 -0.01554432
  -0.05075077  0.26004148  0.06108981  0.01424543 -0.3146082   0.21298297
  -0.17215993 -0.16199788 -0.19881345